<a href="https://colab.research.google.com/github/abhigyanpal1/PalAbhigyan-s-Neural-Network-PANN-/blob/main/Fine_Tuning_LLMs_with_Hugging_Face_Partial_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning LLMs with Hugging Face

## Step 1: Installing and importing the libraries

In [1]:
!pip uninstall accelerate peft bitsandbytes transformers trl -y
!pip install accelerate peft==0.13.2 bitsandbytes transformers trl==0.12.0

Found existing installation: accelerate 1.2.1
Uninstalling accelerate-1.2.1:
  Successfully uninstalled accelerate-1.2.1
Found existing installation: peft 0.14.0
Uninstalling peft-0.14.0:
  Successfully uninstalled peft-0.14.0
Found existing installation: transformers 4.47.1
Uninstalling transformers-4.47.1:
  Successfully uninstalled transformers-4.47.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
 

In [2]:
!pip install huggingface_hub

In [3]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

## Step 2: Loading the model

In [4]:
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True,
                                                                                            bnb_4bit_compute_dtype = getattr(torch, "float16"),
                                                                                            bnb_4bit_quant_type = "nf4") ) # automodel for causal will predict words based on the previous words # quantization config is used here to reduce the model size by applying 4 bit precision # bnb_compute_4bit_dtype mein it will fetch the best datatype for computations # bnb_4bit_quant_type yaha pe fp4 ki jagah nf4 use kiya hai
llama_model.config.use_cache = False # outputs of previously computed layers is not stored in the cache which reduces the size of the model and speeds up the training computations
llama_model.config.pretraining_tp = 1 # deactivate the more accurate computations of the linear layers because if it was kept activated, it would slow down the linear layers computations


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

## Step 3: Loading the tokenizer

In [5]:
# tokenizer must be compatible with the llama 2 model
# tokenizer must use the same tokens as the model and the padding also must be the same
# a tokenizer converts the inputs into a format that can be processed by a model
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token # padding is used to make the lenght of every sequence same, here, we want the pad token to be equal to the end of sequence token(eos means end of sequence)
# the sequences will get filled with the end of string token and hence, the padding side should be right
llama_tokenizer.padding_side = "right" # padding must be done from the right side

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

## Step 4: Setting the training arguments

In [6]:
# configure the training parameters of the future training that will happen once the llama-2 model is retrained using medical data
# training arguments class is from the transformers library
# output_dir -> The output directory where the model predictions and checkpoints will be written
# per_device_train_batch_size = 4 -> the model will process 4 training examples in each training
# max_steps = 100 -> limits the training to maximum 100 steps. For a finite dataset, training is reiterated through the dataset (if all data is exhausted) until max_steps is reached
training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    max_steps=100,
    run_name="Abhi",
    gradient_checkpointing=True,
    save_steps=10,  # Save checkpoints every 10 steps
    save_total_limit=2,  # Keep only the 2 most recent checkpoints
)


## Step 5: Creating the Supervised Fine-Tuning trainer

In [7]:
# supervised fine-tuning is a transfer learning technique that updates the weights of a pretrained model based on the new data provided
# Parameter efficient fine-tuning is used here to reduce the number of parameters. Here LoRA will be used
# LoRA (Low-Rank Adaptation of Large Language Models) is a popular and lightweight training technique that significantly reduces the number of trainable parameters.
llama_sft_trainer = SFTTrainer(model = llama_model,
                               args = training_arguments,
                               train_dataset = load_dataset(path = "aboonaji/wiki_medical_terms_llam2_format", split = "train"),
                               tokenizer = llama_tokenizer,
                               peft_config = LoraConfig(task_type = "CAUSAL_LM", r = 64, lora_alpha = 16, lora_dropout = 0.1),
                               dataset_text_field = "text")
# task_type = "CAUSAL_LM" -> task_type is CAUSAL_LM for causal language modelling
# r = 64 We can say that, as the rank increases, LORA essentially converges toward normal fine-tuning. A lower rank means fewer trainable parameters, resulting in less memory required for fine-tuning the model
# lora_alpha = 16 -> scaling factor for weight matrices
# lora_dropout = 0.1 -> deactivating some of the parameters during the training so that they are not updated all the time and it prevents overfitting
# dataset_text_field = "text" -> input to the model will be text


wiki_medical_terms_llam2.jsonl:   0%|          | 0.00/54.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6861 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/6861 [00:00<?, ? examples/s]

## Step 6: Training the model

In [9]:
import os

# Disable wandb logging
os.environ["WANDB_MODE"] = "disabled"

# Start training
llama_sft_trainer.train()


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

TrainOutput(global_step=100, training_loss=1.7008706665039062, metrics={'train_runtime': 2222.2607, 'train_samples_per_second': 0.09, 'train_steps_per_second': 0.045, 'total_flos': 7178146700083200.0, 'train_loss': 1.7008706665039062, 'epoch': 0.02914602156805596})

## Step 7: Chatting with the model

In [11]:
user_prompt = input()
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 300)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

Please tell me about Ascariasis


Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>[INST] Please tell me about Ascariasis [/INST]  Ascariasis is a parasitic infection caused by the Ascaris lumbricoides roundworm, which is the most common intestinal parasite worldwide. hopefully, this article will provide you with a comprehensive overview of ascariasis, including its definition, causes, symptoms, diagnosis, treatment, and prevention.

Definition:

Ascariasis is a parasitic infection caused by the Ascaris lumbricoides roundworm, which is the most common intestinal parasite worldwide. The infection occurs when the worms migrate from the small intestine to other organs in the body, such as the liver, lungs, and brain.

Causes:

Ascariasis is caused by the ingestion of infective larvae of the Ascaris lumbricoides roundworm. The infection can occur through contaminated food or water, direct contact with an infected person's feces, or through penetration of the skin.

Symptoms:

The symptoms of ascariasis can vary depending on the severity of the infection and the organs